In [21]:
import pandas as pd
import numpy as np
import pickle, os
from dateutil import parser


In [6]:
mimiciv_dir = '../mimic_iv/'
cohort_dir = '../phase1_teamA/'

In [206]:
predischg_interval_hrs = 48
delta_interval_hrs = 24
rare_med_threshold = 10

## Load Data

In [11]:
cohort = pd.read_csv(f'{cohort_dir}final_cohort_selected.csv', low_memory=False)
cohort_ids = set(cohort.hadm_id.unique())

In [24]:
all_admissions = pd.read_csv(f'{mimiciv_dir}core/admissions.csv', low_memory=False)
selected_admissions = all_admissions[all_admissions.hadm_id.isin(cohort_ids)]

In [14]:
d_labitems = pd.read_csv(f'{mimiciv_dir}hosp/d_labitems.csv', low_memory=False)

In [15]:
iter_emar = pd.read_csv(f'{mimiciv_dir}hosp/emar.csv', iterator=True, chunksize=100000)
selected_emars = pd.concat([chunk[chunk.hadm_id.isin(cohort_ids)] for chunk in iter_emar])

In [16]:
iter_labs = pd.read_csv(f'{mimiciv_dir}hosp/labevents.csv', iterator=True, chunksize=100000)
selected_labevs = pd.concat([chunk[chunk.hadm_id.isin(cohort_ids)] for chunk in iter_labs])

/Users/jmcculloch/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/jmcculloch/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/jmcculloch/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/jmcculloch/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3338: DtypeWarning: Columns (9,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, resu

## Admission and Discharge

In [32]:
admittimes = selected_admissions['admittime'].map(parser.parse)
dischtimes = selected_admissions['dischtime'].map(parser.parse)
stay_duration = (dischtimes - admittimes) / np.timedelta64(1, 'h')

In [253]:
# intialise data of lists.
features_data = {'hadm_id': selected_admissions.hadm_id.to_numpy(),
        'stay_duration_hrs': stay_duration.to_numpy(),
        'admittime': admittimes,
         'dischtime': dischtimes};
  
# Create DataFrame
features = pd.DataFrame(features_data)
features = features.set_index('hadm_id')

In [176]:
features.head()

,stay_duration_hrs,admittime,dischtime
hadm_id,,,
26732088,24.433333,2131-09-04 14:26:00,2131-09-05 14:52:00
25141335,19.133333,2186-10-18 17:52:00,2186-10-19 13:00:00
23337747,25.166667,2134-08-11 16:06:00,2134-08-12 17:16:00
26726555,42.083333,2135-05-26 19:25:00,2135-05-28 13:30:00
25401613,50.700000,2123-10-18 07:15:00,2123-10-20 09:57:00


## Medications

In [256]:
med_list = selected_emars.medication.value_counts() # TODO switch to using training set only

In [258]:
first_rare = np.argmax(med_list.to_numpy() < rare_med_threshold)
meds = [k for k in med_dict.keys()] # Get list of meds
for i in range(first_rare): # For nonrare meds, get which hadms have the med
    hadms = set(selected_emars.hadm_id[selected_emars.medication == meds[i]].unique())
    features[meds[i]] = features.index.isin(hadms)

rare_meds = set([meds[i] for i in range(first_rare, len(meds))])
hadms = set(selected_emars.hadm_id[selected_emars.medication.isin(rare_meds)].unique())
features["rare_medication"] = features.index.isin(hadms)

In [91]:
features.head()

,stay_duration_hrs,Sodium Chloride 0.9% Flush,Insulin,Heparin,Furosemide,Metoprolol Tartrate,Docusate Sodium,Acetaminophen,Ipratropium-Albuterol Neb,Aspirin,...,B-12 Plus,Invirase,RISperidone (Disintegrating Tablet),Alphagan P,Hydrocortisone study med,Butrans,Vancomycin (0.5 mg/mL)-Heparin (100 units/mL) Lock,LaMICtal XR,alfuzosin,rare_medication
hadm_id,,,,,,,,,,,,,,,,,,,,,
26732088,24.433333,True,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25141335,19.133333,True,True,False,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
23337747,25.166667,True,False,False,True,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
26726555,42.083333,True,False,False,True,True,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
25401613,50.700000,True,True,False,True,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


## Labs

In [283]:
d_labitems[d_labitems.label.str.contains("lactate", case=False, na=False)]


,itemid,label,fluid,category,loinc_code
938,50813,Lactate,Blood,Blood Gas,NaN
939,52442,Lactate,Blood,Blood Gas,NaN
940,50843,"Lactate Dehydrogenase, Ascites",Ascites,Chemistry,NaN
941,51795,"Lactate Dehydrogenase, CSF",Cerebrospinal Fluid,Chemistry,NaN
942,50954,Lactate Dehydrogenase (LD),Blood,Chemistry,2532-0
943,51054,"Lactate Dehydrogenase, Pleural",Pleural,Chemistry,NaN
944,51944,"Lactate Dehydrogenase, Stool",Stool,Chemistry,NaN


In [284]:
lab_itemids = [50862, 51006, 50893, 50882, 50902, 50912, 50931, 50971, 50983, 50885, 50976, 50861, 50863, 50878,#CMP
                51265, 51279, 51301, #CBC 
                50954, 51274, #LFTs
                51237, #INR
                51100, 51078, 51097,#Urine Studies
                50910, 50956, 51288, 50889, #Injury/Inflammatory markers
                50963 # BNP
              ]
lab_itemnames = d_labitems[d_labitems.itemid.isin(lab_itemids)].label.to_numpy()
print(lab_itemnames)

['Alanine Aminotransferase (ALT)' 'Albumin' 'Alkaline Phosphatase'
 'Asparate Aminotransferase (AST)' 'Bicarbonate' 'Bilirubin, Total'
 'Calcium, Total' 'Chloride' 'Chloride, Urine' 'C-Reactive Protein'
 'Creatine Kinase (CK)' 'Creatinine' 'Glucose' 'INR(PT)'
 'Lactate Dehydrogenase (LD)' 'Lipase' 'NTproBNP' 'Platelet Count'
 'Potassium' 'Potassium, Urine' 'Protein, Total' 'PT' 'Red Blood Cells'
 'Sedimentation Rate' 'Sodium' 'Sodium, Urine' 'Urea Nitrogen'
 'White Blood Cells']


In [275]:
(selected_labevs.itemid == 50889).sum()

1470

In [172]:
selected_labevs['charttime'] = pd.to_datetime(selected_labevs['charttime'])

In [285]:
for itemid in lab_itemids:
    # Get lab name
    item_name = d_labitems[d_labitems.itemid == itemid].label.to_numpy()[0]
    features.reset_index()
    # Get first and last lab in stay 
    labs = selected_labevs.loc[(selected_labevs.itemid==itemid) & ~selected_labevs.valuenum.isnull()][['hadm_id','charttime','valuenum']]
    labs = labs.merge(features, how='left', left_on= 'hadm_id', right_on= 'hadm_id')
    hrs_before_disch = (labs.dischtime - labs.charttime)/ np.timedelta64(1, 'h')
    labs = labs[(hrs_before_disch > 0) & (hrs_before_disch < labs.stay_duration_hrs)]
    latest_labs = labs.loc[labs.groupby('hadm_id').charttime.idxmax()].set_index('hadm_id')
    initial_labs = labs.loc[labs.groupby('hadm_id').charttime.idxmin()].set_index('hadm_id')
    features = features.join(latest_labs[['charttime', 'valuenum']].add_suffix("_latest"), how='outer')
    features = features.join(initial_labs[['charttime', 'valuenum']].add_suffix("_init"), how='outer')
    features[str(item_name) + " Latest"] = features.valuenum_latest
    delta_valid =  (features.charttime_latest - features.charttime_init) / np.timedelta64(1, 'h') > delta_interval_hrs
    features[str(item_name) + " Delta"] = (features.valuenum_latest - features.valuenum_init)[delta_valid]
    features = features.drop(['charttime_latest', 'charttime_init', 'valuenum_latest', 'valuenum_init'], axis=1)
features.head()

,stay_duration_hrs,admittime,dischtime,Albumin Latest,Albumin Delta,Urea Nitrogen Latest,Urea Nitrogen Delta,"Calcium, Total Latest","Calcium, Total Delta",Bicarbonate Latest,...,"Potassium, Urine Latest","Potassium, Urine Delta",Creatine Kinase (CK) Latest,Creatine Kinase (CK) Delta,Lipase Latest,Lipase Delta,Sedimentation Rate Latest,Sedimentation Rate Delta,C-Reactive Protein Latest,C-Reactive Protein Delta
hadm_id,,,,,,,,,,,,,,,,,,,,,
20001395,209.966667,2180-04-23 22:52:00,2180-05-02 16:50:00,NaN,NaN,37.0,19.0,9.6,0.3,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20002497,262.150000,2121-08-19 19:33:00,2121-08-30 17:42:00,NaN,NaN,22.0,6.0,9.4,0.8,30.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20003491,252.650000,2197-12-18 04:50:00,2197-12-28 17:29:00,3.0,NaN,41.0,0.0,8.7,0.5,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20003739,220.716667,2183-10-06 12:41:00,2183-10-15 17:24:00,NaN,NaN,16.0,-14.0,8.9,-1.0,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20003880,213.050000,2187-09-16 21:52:00,2187-09-25 18:55:00,2.8,0.1,60.0,-23.0,8.1,-0.4,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [249]:
features.head()

,stay_duration_hrs,admittime,dischtime
hadm_id,,,
26732088,24.433333,2131-09-04 14:26:00,2131-09-05 14:52:00
25141335,19.133333,2186-10-18 17:52:00,2186-10-19 13:00:00
23337747,25.166667,2134-08-11 16:06:00,2134-08-12 17:16:00
26726555,42.083333,2135-05-26 19:25:00,2135-05-28 13:30:00
25401613,50.700000,2123-10-18 07:15:00,2123-10-20 09:57:00


## Save features to file

In [286]:
features.to_csv('features.csv')